# 0. Import Dependencies

In [51]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import tensorflow as tf
import math

from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
from tensorflow.keras.utils import to_categorical

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

from tensorflow.keras.models import Sequential, Model

from tensorflow.keras.layers import (LSTM, Dense, Concatenate, Attention, Dropout, Softmax,
                                     Input, Flatten, Activation, Bidirectional, Permute, multiply, 
                                     ConvLSTM2D, MaxPooling3D, TimeDistributed, Conv2D, MaxPooling2D)

from scipy import stats

# 1. Keypoints using MP Pose

In [52]:
# Pre-trained pose estimation model from Google Mediapipe
mp_pose = mp.solutions.pose

# Supported Mediapipe visualization tools
mp_drawing = mp.solutions.drawing_utils

In [53]:
def mediapipe_detection(image, model):
    """
    This function detects human pose estimation keypoints from webcam footage
    
    """
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [54]:
def draw_landmarks(image, results):
    """
    This function draws keypoints and landmarks detected by the human pose estimation model
    
    """
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )

In [55]:
cap = cv2.VideoCapture(0) # camera object
HEIGHT = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # webcam video frame height
WIDTH = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # webcam video frame width
FPS = int(cap.get(cv2.CAP_PROP_FPS)) # webcam video fram rate 

# Set and test mediapipe model using webcam
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()
      
        # Make detection
        image, results = mediapipe_detection(frame, pose)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass
        
        # Render detections
        draw_landmarks(image, results)               
        
        # Display frame on screen
        cv2.imshow('OpenCV Feed', image)
        
        # Exit / break out logic
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# 2. Extract Keypoints

In [56]:
# Recollect and organize keypoints from the test
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [57]:
# 33 landmarks with 4 values (x, y, z, visibility)
num_landmarks = len(landmarks)
num_values = len(test)
num_input_values = num_landmarks*num_values

In [58]:
# This is an example of what we would use as an input into our AI models
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)

In [59]:
def extract_keypoints(results):
    """
    Processes and organizes the keypoints detected from the pose estimation model 
    to be used as inputs for the exercise decoder models
    
    """
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    return pose

# 3. Setup Folders for Collection

In [60]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join(os. getcwd(),'data') 
print(DATA_PATH)

# make directory if it does not exist yet
if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

# Actions/exercises that we try to detect
actions = np.array(['curl', 'press', 'squat'])
num_classes = len(actions)

# How many videos worth of data
no_sequences = 50

# Videos are going to be this many frames in length
sequence_length = FPS*1

# Folder start
# Change this to collect more data and not lose previously collected data
start_folder = 101

C:\Users\cpras\Documents\GitHub\AI_Personal_Trainer\data


In [62]:
# Build folder paths
for action in actions:     
    for sequence in range(start_folder,no_sequences+start_folder):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))  
        except:
            pass

# 4. Collect Keypoint Values for Training and Testing

In [63]:
# Colors associated with each exercise (e.g., curls are denoted by blue, squats are denoted by orange, etc.)
colors = [(245,117,16), (117,245,16), (16,117,245)]

In [64]:
# Collect Training Data

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    # Loop through actions
    for idx, action in enumerate(actions):
        # Loop through sequences (i.e., videos)
        for sequence in range(start_folder, start_folder+no_sequences):
            # Loop through video length (i.e, sequence length)
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()
                
                # Make detection
                image, results = mediapipe_detection(frame, pose)

                # Extract landmarks
                try:
                    landmarks = results.pose_landmarks.landmark
                except:
                    pass
                
                # Render detections
                draw_landmarks(image, results) 

                # Apply visualization logic
                if frame_num == 0: # If first frame in sequence, print that you're starting a new data collection and wait 500 ms
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    
                    cv2.putText(image, 'Collecting {} Video # {}'.format(action, sequence), (15,30), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 8, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting {} Video # {}'.format(action, sequence), (15,30), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, colors[idx], 4, cv2.LINE_AA)
                    
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting {} Video # {}'.format(action, sequence), (15,30), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 8, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting {} Video # {}'.format(action, sequence), (15,30), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, colors[idx], 4, cv2.LINE_AA)
                    
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)

                # Export keypoints (sequence + pose landmarks)
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [65]:
cap.release()
cv2.destroyAllWindows()

# 5. Preprocess Data and Create Labels/Features

In [66]:
label_map = {label:num for num, label in enumerate(actions)}

In [67]:
# Load and organize recorded training data
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):         
            # LSTM input data
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)  
            
        sequences.append(window)
        labels.append(label_map[action])

In [68]:
# Make sure first dimensions of arrays match
X = np.array(sequences)
y = to_categorical(labels).astype(int)
print(X.shape, y.shape)

(450, 30, 132) (450, 3)


In [69]:
# Split into training, validation, and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1)
print(X_train.shape, y_train.shape)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=15/90, random_state=2)

(405, 30, 132) (405, 3)


# 6. Build and Train Neural Networks

In [78]:
# Callbacks to be used during neural network training 
es_callback = EarlyStopping(monitor='val_loss', patience=10, verbose=0)
lr_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001, verbose=0)
chkpt_callback = ModelCheckpoint(filepath=DATA_PATH, monitor='val_loss', verbose=0, save_best_only=False, 
                                 save_weights_only=False, mode='auto', save_freq=1)

batch_size = 32
max_epochs = 500

## 6a. LSTM

In [71]:
# Set up Tensorboard logging and callbacks
NAME = f"ExerciseRecognition-LSTM-{int(time.time())}"
log_dir = os.path.join(os.getcwd(), 'logs', NAME,'')
tb_callback = TensorBoard(log_dir=log_dir)

callbacks = [tb_callback, es_callback, lr_callback, chkpt_callback]

In [72]:
lstm = Sequential()
lstm.add(LSTM(128, return_sequences=True, activation='relu', input_shape=(sequence_length, num_input_values)))
lstm.add(LSTM(256, return_sequences=True, activation='relu'))
lstm.add(LSTM(128, return_sequences=False, activation='relu'))
lstm.add(Dense(128, activation='relu'))
lstm.add(Dense(64, activation='relu'))
lstm.add(Dense(actions.shape[0], activation='softmax'))
print(lstm.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 30, 128)           133632    
                                                                 
 lstm_5 (LSTM)               (None, 30, 256)           394240    
                                                                 
 lstm_6 (LSTM)               (None, 128)               197120    
                                                                 
 dense_6 (Dense)             (None, 128)               16512     
                                                                 
 dense_7 (Dense)             (None, 64)                8256      
                                                                 
 dense_8 (Dense)             (None, 3)                 195       
                                                                 
Total params: 749,955
Trainable params: 749,955
Non-tr

In [73]:
lstm.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
lstm.fit(X_train, y_train, batch_size=batch_size, epochs=max_epochs, validation_data=(X_val, y_val), callbacks=callbacks)

Epoch 1/500
INFO:tensorflow:Assets written to: C:\Users\cpras\Documents\GitHub\AI_Personal_Trainer\data\assets
11/11 [==============================] - 55s 5s/step - loss: 0.9920 - categorical_accuracy: 0.4777 - val_loss: 0.8348 - val_categorical_accuracy: 0.5735 - lr: 0.0010
Epoch 2/500
INFO:tensorflow:Assets written to: C:\Users\cpras\Documents\GitHub\AI_Personal_Trainer\data\assets
11/11 [==============================] - 46s 4s/step - loss: 0.6790 - categorical_accuracy: 0.6884 - val_loss: 0.5039 - val_categorical_accuracy: 0.6765 - lr: 0.0010
Epoch 3/500
INFO:tensorflow:Assets written to: C:\Users\cpras\Documents\GitHub\AI_Personal_Trainer\data\assets
11/11 [==============================] - 53s 5s/step - loss: 0.4066 - categorical_accuracy: 0.7864 - val_loss: 0.3969 - val_categorical_accuracy: 0.7941 - lr: 0.0010
Epoch 4/500
INFO:tensorflow:Assets written to: C:\Users\cpras\Documents\GitHub\AI_Personal_Trainer\data\assets
 7/11 [==================>...........] - ETA: 19s - loss: 

11/11 [==============================] - 56s 5s/step - loss: 0.2709 - categorical_accuracy: 0.9318 - val_loss: 0.0896 - val_categorical_accuracy: 0.9706 - lr: 0.0010
Epoch 5/500
INFO:tensorflow:Assets written to: C:\Users\cpras\Documents\GitHub\AI_Personal_Trainer\data\assets
11/11 [==============================] - 46s 4s/step - loss: 0.9178 - categorical_accuracy: 0.5460 - val_loss: 0.8063 - val_categorical_accuracy: 0.5882 - lr: 0.0010
Epoch 6/500
INFO:tensorflow:Assets written to: C:\Users\cpras\Documents\GitHub\AI_Personal_Trainer\data\assets
11/11 [==============================] - 52s 5s/step - loss: 0.8078 - categorical_accuracy: 0.8398 - val_loss: 1.0893 - val_categorical_accuracy: 0.4412 - lr: 0.0010
Epoch 7/500
INFO:tensorflow:Assets written to: C:\Users\cpras\Documents\GitHub\AI_Personal_Trainer\data\assets
11/11 [==============================] - 45s 4s/step - loss: 0.7629 - categorical_accuracy: 0.6202 - val_loss: 0.7856 - val_categorical_accuracy: 0.5294 - lr: 0.0010
Epo

11/11 [==============================] - 46s 4s/step - loss: 0.7536 - categorical_accuracy: 0.6588 - val_loss: 0.6995 - val_categorical_accuracy: 0.6765 - lr: 0.0010
Epoch 9/500
INFO:tensorflow:Assets written to: C:\Users\cpras\Documents\GitHub\AI_Personal_Trainer\data\assets
11/11 [==============================] - 45s 4s/step - loss: 0.6273 - categorical_accuracy: 0.7834 - val_loss: 0.6081 - val_categorical_accuracy: 0.6176 - lr: 0.0010
Epoch 10/500
INFO:tensorflow:Assets written to: C:\Users\cpras\Documents\GitHub\AI_Personal_Trainer\data\assets
11/11 [==============================] - 49s 4s/step - loss: 0.4864 - categorical_accuracy: 0.7151 - val_loss: 0.4717 - val_categorical_accuracy: 0.6765 - lr: 2.0000e-04
Epoch 11/500
INFO:tensorflow:Assets written to: C:\Users\cpras\Documents\GitHub\AI_Personal_Trainer\data\assets
11/11 [==============================] - 55s 5s/step - loss: 0.3418 - categorical_accuracy: 0.8427 - val_loss: 0.2394 - val_categorical_accuracy: 0.9118 - lr: 2.00

11/11 [==============================] - 59s 5s/step - loss: 0.1365 - categorical_accuracy: 0.9614 - val_loss: 0.0597 - val_categorical_accuracy: 0.9853 - lr: 2.0000e-04
Epoch 13/500
INFO:tensorflow:Assets written to: C:\Users\cpras\Documents\GitHub\AI_Personal_Trainer\data\assets
11/11 [==============================] - 52s 5s/step - loss: 0.0737 - categorical_accuracy: 0.9763 - val_loss: 0.0221 - val_categorical_accuracy: 1.0000 - lr: 2.0000e-04
Epoch 14/500
INFO:tensorflow:Assets written to: C:\Users\cpras\Documents\GitHub\AI_Personal_Trainer\data\assets
11/11 [==============================] - 48s 4s/step - loss: 0.0831 - categorical_accuracy: 0.9733 - val_loss: 0.0821 - val_categorical_accuracy: 0.9706 - lr: 2.0000e-04
Epoch 15/500
INFO:tensorflow:Assets written to: C:\Users\cpras\Documents\GitHub\AI_Personal_Trainer\data\assets
 8/11 [====================>.........] - ETA: 12s - loss: 0.0789 - categorical_accuracy: 0.9766INFO:tensorflow:Assets written to: C:\Users\cpras\Documents

11/11 [==============================] - 47s 4s/step - loss: 0.0756 - categorical_accuracy: 0.9733 - val_loss: 0.0538 - val_categorical_accuracy: 1.0000 - lr: 2.0000e-04
Epoch 16/500
INFO:tensorflow:Assets written to: C:\Users\cpras\Documents\GitHub\AI_Personal_Trainer\data\assets
11/11 [==============================] - 60s 6s/step - loss: 0.0447 - categorical_accuracy: 0.9941 - val_loss: 0.0169 - val_categorical_accuracy: 1.0000 - lr: 2.0000e-04
Epoch 17/500
INFO:tensorflow:Assets written to: C:\Users\cpras\Documents\GitHub\AI_Personal_Trainer\data\assets
11/11 [==============================] - 47s 4s/step - loss: 0.0178 - categorical_accuracy: 0.9970 - val_loss: 0.0029 - val_categorical_accuracy: 1.0000 - lr: 2.0000e-04
Epoch 18/500
INFO:tensorflow:Assets written to: C:\Users\cpras\Documents\GitHub\AI_Personal_Trainer\data\assets
11/11 [==============================] - 47s 4s/step - loss: 0.0097 - categorical_accuracy: 0.9970 - val_loss: 9.6637e-04 - val_categorical_accuracy: 1.00

11/11 [==============================] - 46s 4s/step - loss: 0.0077 - categorical_accuracy: 0.9970 - val_loss: 4.6912e-04 - val_categorical_accuracy: 1.0000 - lr: 2.0000e-04
Epoch 20/500
INFO:tensorflow:Assets written to: C:\Users\cpras\Documents\GitHub\AI_Personal_Trainer\data\assets
11/11 [==============================] - 50s 5s/step - loss: 0.0063 - categorical_accuracy: 0.9970 - val_loss: 4.5941e-04 - val_categorical_accuracy: 1.0000 - lr: 2.0000e-04
Epoch 21/500
INFO:tensorflow:Assets written to: C:\Users\cpras\Documents\GitHub\AI_Personal_Trainer\data\assets
11/11 [==============================] - 52s 5s/step - loss: 0.0060 - categorical_accuracy: 0.9970 - val_loss: 0.0014 - val_categorical_accuracy: 1.0000 - lr: 2.0000e-04
Epoch 22/500
INFO:tensorflow:Assets written to: C:\Users\cpras\Documents\GitHub\AI_Personal_Trainer\data\assets
11/11 [==============================] - 54s 5s/step - loss: 0.0218 - categorical_accuracy: 0.9941 - val_loss: 0.0015 - val_categorical_accuracy: 

11/11 [==============================] - 52s 5s/step - loss: 0.0288 - categorical_accuracy: 0.9911 - val_loss: 0.0093 - val_categorical_accuracy: 1.0000 - lr: 2.0000e-04
Epoch 24/500
INFO:tensorflow:Assets written to: C:\Users\cpras\Documents\GitHub\AI_Personal_Trainer\data\assets
11/11 [==============================] - 52s 5s/step - loss: 0.0150 - categorical_accuracy: 0.9970 - val_loss: 0.0026 - val_categorical_accuracy: 1.0000 - lr: 2.0000e-04
Epoch 25/500
INFO:tensorflow:Assets written to: C:\Users\cpras\Documents\GitHub\AI_Personal_Trainer\data\assets
11/11 [==============================] - 53s 5s/step - loss: 0.0074 - categorical_accuracy: 0.9970 - val_loss: 0.0023 - val_categorical_accuracy: 1.0000 - lr: 4.0000e-05
Epoch 26/500
INFO:tensorflow:Assets written to: C:\Users\cpras\Documents\GitHub\AI_Personal_Trainer\data\assets
 8/11 [====================>.........] - ETA: 14s - loss: 0.0072 - categorical_accuracy: 0.9961INFO:tensorflow:Assets written to: C:\Users\cpras\Documents

11/11 [==============================] - 52s 5s/step - loss: 0.0065 - categorical_accuracy: 0.9970 - val_loss: 0.0020 - val_categorical_accuracy: 1.0000 - lr: 4.0000e-05
Epoch 27/500
INFO:tensorflow:Assets written to: C:\Users\cpras\Documents\GitHub\AI_Personal_Trainer\data\assets
11/11 [==============================] - 50s 5s/step - loss: 0.0059 - categorical_accuracy: 0.9970 - val_loss: 0.0017 - val_categorical_accuracy: 1.0000 - lr: 4.0000e-05
Epoch 28/500
INFO:tensorflow:Assets written to: C:\Users\cpras\Documents\GitHub\AI_Personal_Trainer\data\assets
11/11 [==============================] - 53s 5s/step - loss: 0.0054 - categorical_accuracy: 0.9970 - val_loss: 0.0014 - val_categorical_accuracy: 1.0000 - lr: 4.0000e-05
Epoch 29/500
INFO:tensorflow:Assets written to: C:\Users\cpras\Documents\GitHub\AI_Personal_Trainer\data\assets
11/11 [==============================] - 50s 5s/step - loss: 0.0048 - categorical_accuracy: 0.9970 - val_loss: 0.0012 - val_categorical_accuracy: 1.0000 -

11/11 [==============================] - 47s 4s/step - loss: 0.0046 - categorical_accuracy: 0.9970 - val_loss: 0.0012 - val_categorical_accuracy: 1.0000 - lr: 1.0000e-05


## 6b. LSTM + Attention

In [79]:
# Set up Tensorboard logging and callbacks
NAME = f"ExerciseRecognition-AttnLSTM-{int(time.time())}"
log_dir = os.path.join(os.getcwd(), 'logs', NAME,'')
tb_callback = TensorBoard(log_dir=log_dir)

callbacks = [tb_callback, es_callback, lr_callback, chkpt_callback]

In [80]:
def attention_block(inputs, time_steps):
    """
    Attention layer for deep neural network
    
    """
    # Attention weights
    a = Permute((2, 1))(inputs)
    a = Dense(time_steps, activation='softmax')(a)
    
    # Attention vector
    a_probs = Permute((2, 1), name='attention_vec')(a)
    
    # Luong's multiplicative score
    output_attention_mul = multiply([inputs, a_probs], name='attention_mul') 
    
    return output_attention_mul

In [81]:
HIDDEN_UNITS = 256

# Input
inputs = Input(shape=(sequence_length, num_input_values))

# Bi-LSTM
lstm_out = Bidirectional(LSTM(HIDDEN_UNITS, return_sequences=True))(inputs)

# Attention
attention_mul = attention_block(lstm_out, sequence_length)
attention_mul = Flatten()(attention_mul)

# Fully Connected Layer
x = Dense(2*HIDDEN_UNITS, activation='relu')(attention_mul)
x = Dropout(0.5)(x)

# Output
x = Dense(actions.shape[0], activation='softmax')(x)

# Bring it all together
AttnLSTM = Model(inputs=[inputs], outputs=x)
print(AttnLSTM.summary())

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 30, 132)]    0           []                               
                                                                                                  
 bidirectional_2 (Bidirectional  (None, 30, 512)     796672      ['input_3[0][0]']                
 )                                                                                                
                                                                                                  
 permute_2 (Permute)            (None, 512, 30)      0           ['bidirectional_2[0][0]']        
                                                                                                  
 dense_12 (Dense)               (None, 512, 30)      930         ['permute_2[0][0]']        

In [82]:
AttnLSTM.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
AttnLSTM.fit(X_train, y_train, batch_size=batch_size, epochs=max_epochs, validation_data=(X_val, y_val), callbacks=callbacks)

Epoch 1/500


INFO:tensorflow:Assets written to: C:\Users\cpras\Documents\GitHub\AI_Personal_Trainer\data\assets


INFO:tensorflow:Assets written to: C:\Users\cpras\Documents\GitHub\AI_Personal_Trainer\data\assets


 1/11 [=>............................] - ETA: 2:19 - loss: 1.0973 - categorical_accuracy: 0.3750

INFO:tensorflow:Assets written to: C:\Users\cpras\Documents\GitHub\AI_Personal_Trainer\data\assets


...

 1/11 [=>............................] - ETA: 2:18 - loss: 6.4454e-05 - categorical_accuracy: 1.0000

INFO:tensorflow:Assets written to: C:\Users\cpras\Documents\GitHub\AI_Personal_Trainer\data\assets


INFO:tensorflow:Assets written to: C:\Users\cpras\Documents\GitHub\AI_Personal_Trainer\data\assets


 2/11 [====>.........................] - ETA: 1:57 - loss: 6.9168e-05 - categorical_accuracy: 1.0000

KeyboardInterrupt: 

In [83]:
# Model map
models = {
    'LSTM': lstm, 
    'LSTM_Attention_128HUs': AttnLSTM, 
}

# 7a. Save Weights

In [84]:
for model_name, model in models.items():
    save_dir = os.path.join(os.getcwd(), f"{model_name}.h5")
    model.save(save_dir)

# 7b. Load Weights

In [85]:
# Run model rebuild before doing this
for model_name, model in models.items():
    load_dir = os.path.join(os.getcwd(), f"{model_name}.h5")
    model.load_weights(load_dir)

# 8. Make Predictions

In [86]:
for model in models.values():
    res = model.predict(X_test, verbose=0)   

# 9. Evaluations using Confusion Matrix and Accuracy

In [87]:
eval_results = {}
eval_results['confusion matrix'] = None
eval_results['Classification Accuracy'] = None

confusion_matrices = {}
classification_accuracies = {}

for model_name, model in models.items():
    yhat = model.predict(X_test, verbose=0)
    
    # Get list of classification predictions
    ytrue = np.argmax(y_test, axis=1).tolist()
    yhat = np.argmax(yhat, axis=1).tolist()
    
    # Confusion matrix
    confusion_matrices[model_name] = multilabel_confusion_matrix(ytrue, yhat)
    
    # Model accuracy
    classification_accuracies[model_name] = accuracy_score(ytrue, yhat)    
    print(f"{model_name} classification accuracy = {classification_accuracies[model_name]*100}%")

# Collect results 
eval_results['confusion matrix'] = confusion_matrices
eval_results['classification accuracy'] = classification_accuracies
    

LSTM classification accuracy = 97.77777777777777%
LSTM_Attention_128HUs classification accuracy = 100.0%


# 10. Choose Model to Test in Real Time

In [88]:
model = AttnLSTM
model_name = 'AttnLSTM'

# 11. Calculate Joint Angles & Count Reps

In [89]:
def calculate_angle(a,b,c):
    """
    Computes 3D joint angle inferred by 3 keypoints and their relative positions to one another
    
    """
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [90]:
def get_coordinates(landmarks, mp_pose, side, joint):
    """
    Retrieves x and y coordinates of a particular keypoint from the pose estimation model
         
     Args:
         landmarks: processed keypoints from the pose estimation model
         mp_pose: Mediapipe pose estimation model
         side: 'left' or 'right'. Denotes the side of the body of the landmark of interest.
         joint: 'shoulder', 'elbow', 'wrist', 'hip', 'knee', or 'ankle'. Denotes which body joint is associated with the landmark of interest.
    
    """
    coord = getattr(mp_pose.PoseLandmark,side.upper()+"_"+joint.upper())
    x_coord_val = landmarks[coord.value].x
    y_coord_val = landmarks[coord.value].y
    return [x_coord_val, y_coord_val]            

In [91]:
def viz_joint_angle(image, angle, joint):
    """
    Displays the joint angle value near the joint within the image frame
    
    """
    cv2.putText(image, str(int(angle)), 
                   tuple(np.multiply(joint, [640, 480]).astype(int)), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                        )
    return

In [95]:
def count_reps(image, current_action, landmarks, mp_pose):
    """
    Counts repetitions of each exercise. Global count and stage (i.e., state) variables are updated within this function.
    
    """

    global curl_counter, press_counter, squat_counter, curl_stage, press_stage, squat_stage
    
    if current_action == 'curl':
        # Get coords
        shoulder = get_coordinates(landmarks, mp_pose, 'left', 'shoulder')
        elbow = get_coordinates(landmarks, mp_pose, 'left', 'elbow')
        wrist = get_coordinates(landmarks, mp_pose, 'left', 'wrist')
        
        # calculate elbow angle
        angle = calculate_angle(shoulder, elbow, wrist)
        
        # curl counter logic
        if angle < 30:
            curl_stage = "up" 
        if angle > 140 and curl_stage =='up':
            curl_stage="down"  
            curl_counter +=1
        press_stage = None
        squat_stage = None
            
        # Viz joint angle
        viz_joint_angle(image, angle, elbow)
        
    elif current_action == 'press':
        
        # Get coords
        shoulder = get_coordinates(landmarks, mp_pose, 'left', 'shoulder')
        elbow = get_coordinates(landmarks, mp_pose, 'left', 'elbow')
        wrist = get_coordinates(landmarks, mp_pose, 'left', 'wrist')

        # Calculate elbow angle
        elbow_angle = calculate_angle(shoulder, elbow, wrist)
        
        # Compute distances between joints
        shoulder2elbow_dist = abs(math.dist(shoulder,elbow))
        shoulder2wrist_dist = abs(math.dist(shoulder,wrist))
        
        # Press counter logic
        if (elbow_angle > 130) and (shoulder2elbow_dist < shoulder2wrist_dist):
            press_stage = "up"
        if (elbow_angle < 50) and (shoulder2elbow_dist > shoulder2wrist_dist) and (press_stage =='up'):
            press_stage='down'
            press_counter += 1
        curl_stage = None
        squat_stage = None
            
        # Viz joint angle
        viz_joint_angle(image, elbow_angle, elbow)
        
    elif current_action == 'squat':
        # Get coords
        # left side
        left_shoulder = get_coordinates(landmarks, mp_pose, 'left', 'shoulder')
        left_hip = get_coordinates(landmarks, mp_pose, 'left', 'hip')
        left_knee = get_coordinates(landmarks, mp_pose, 'left', 'knee')
        left_ankle = get_coordinates(landmarks, mp_pose, 'left', 'ankle')
        # right side
        right_shoulder = get_coordinates(landmarks, mp_pose, 'right', 'shoulder')
        right_hip = get_coordinates(landmarks, mp_pose, 'right', 'hip')
        right_knee = get_coordinates(landmarks, mp_pose, 'right', 'knee')
        right_ankle = get_coordinates(landmarks, mp_pose, 'right', 'ankle')
        
        # Calculate knee angles
        left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
        right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
        
        # Calculate hip angles
        left_hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
        right_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
        
        # Squat counter logic
        thr = 165
        if (left_knee_angle < thr) and (right_knee_angle < thr) and (left_hip_angle < thr) and (right_hip_angle < thr):
            squat_stage = "down"
        if (left_knee_angle > thr) and (right_knee_angle > thr) and (left_hip_angle > thr) and (right_hip_angle > thr) and (squat_stage =='down'):
            squat_stage='up'
            squat_counter += 1
        curl_stage = None
        press_stage = None
            
        # Viz joint angles
        viz_joint_angle(image, left_knee_angle, left_knee)
        viz_joint_angle(image, left_hip_angle, left_hip)
        
    else:
        pass

# 12. Test in Real Time

In [96]:
def prob_viz(res, actions, input_frame, colors):
    """
    This function displays the model prediction probability distribution over the set of exercise classes
    as a horizontal bar graph
    
    """
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):        
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [111]:
# 1. New detection variables
sequence = []
predictions = []
res = []
threshold = 0.5 # minimum confidence to classify as an action/exercise
current_action = ''

# Rep counter logic variables
curl_counter = 0
press_counter = 0
squat_counter = 0
curl_stage = None
press_stage = None
squat_stage = None

# Camera object
cap = cv2.VideoCapture(0)

# Video writer object that saves a video of the real time test
fourcc = cv2.VideoWriter_fourcc('M','J','P','G') # video compression format
HEIGHT = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # webcam video frame height
WIDTH = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # webcam video frame width
FPS = int(cap.get(cv2.CAP_PROP_FPS)) # webcam video fram rate 

video_name = os.path.join(os.getcwd(),f"{model_name}_real_time_test.avi")
out = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*"MJPG"), FPS, (WIDTH,HEIGHT))

# Set mediapipe model 
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detection
        image, results = mediapipe_detection(frame, pose)
        
        # Draw landmarks
        draw_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)        
        sequence.append(keypoints)      
        sequence = sequence[-sequence_length:]
              
        if len(sequence) == sequence_length:
            res = model.predict(np.expand_dims(sequence, axis=0), verbose=0)[0]           
            predictions.append(np.argmax(res))
            current_action = actions[np.argmax(res)]
            confidence = np.max(res)
            
        #3. Viz logic
            # Erase current action variable if no probability is above threshold
            if confidence < threshold:
                current_action = ''

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
            # Count reps
            try:
                landmarks = results.pose_landmarks.landmark
                count_reps(
                    image, current_action, landmarks, mp_pose)
            except:
                pass

            # Display graphical information
            cv2.rectangle(image, (0,0), (640, 40), colors[np.argmax(res)], -1)
            cv2.putText(image, 'curl ' + str(curl_counter), (3,30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, 'press ' + str(press_counter), (240,30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, 'squat ' + str(squat_counter), (490,30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
         
        # Show to screen
        cv2.imshow('OpenCV Feed', image)
        
        # Write to video file
        if ret == True:
            out.write(image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [107]:
cap.release()
out.release()
cv2.destroyAllWindows()